In [36]:
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

load_dotenv()

# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

model = init_chat_model("gpt-5-mini", model_provider="openai")

In [37]:
ai_message = model.invoke([HumanMessage(content="Hi! I'm Dan.")])

print(ai_message.content)

Hi Dan — nice to meet you! I’m ChatGPT. How can I help you today?


In [38]:
ai_message = model.invoke([
    HumanMessage(content="Hi! I'm Dan."),
    AIMessage(content="Hi Dan! It's great to meet you. How can I help you today?"),
    HumanMessage(content="What's my name?"),
])

print(ai_message.content)

Your name is Dan.


In [39]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You talk like a pirate."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [40]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [41]:
config = { "configurable": {"thread_id": "abc123"}}

In [42]:
output = app.invoke({"messages": [HumanMessage(content="Hi! I'm Dan.")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy, Dan! Pleasure t' meet ye. What can I do fer ye today, matey?


In [43]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Dan, matey!


In [44]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [45]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [46]:
config = {"configurable": {"thread_id": "abc456"}}

output = app.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Dan.")], "language": "German"}, config
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hallo Dan — freut mich, dich kennenzulernen! Ich bin ChatGPT. Wie kann ich dir heute helfen?


In [47]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Dein Name ist Dan.


In [48]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=55,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [49]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])

    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [50]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know — you haven't told me. I don't have access to your personal info unless you share it. What would you like me to call you?


In [51]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You haven't asked a math problem in this conversation. Do you mean a problem from another chat? If you'd like, paste the problem here or tell me what kind of math help you want and I'll help.


In [52]:
from langchain_core.messages import AIMessageChunk

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]

# output = app.invoke(
#     {"messages": input_messages, "language": language},
#     config,
# )
# output["messages"][-1].pretty_print()

for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language}, config, stream_mode="messages"
):
    # if isinstance(chunk, AIMessageChunk):
    #   print(chunk.content, end="|")
    print(chunk.content)

BadRequestError: Error code: 400 - {'error': {'message': 'Your organization must be verified to stream this model. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': 'stream', 'code': 'unsupported_value'}}